In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/importance-sampling')

In [3]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10, MNIST, CIFAR100
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torchmetrics.functional import accuracy
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
normalize = transforms.Normalize(
    mean=[0.4914, 0.4822, 0.4465],
    std=[0.247, 0.2435, 0.2616],
)

def samplers(n, split_shuffle=True, val_size=0.1):
    if split_shuffle:
        idx = torch.randperm(n, generator=torch.Generator().manual_seed(0))
    else:
        idx = torch.arange(n)
    split_idx = int((1.0 - val_size) * n)
    train_sampler = SubsetRandomSampler(idx[:split_idx])
    val_sampler = SubsetRandomSampler(idx[split_idx:])
    return train_sampler, val_sampler

def train_val_dataloader(root_dir='./cifar10', split_shuffle=True, val_size=0.1, batch_size=120):
    train_dataset = CIFAR10(root=root_dir, train=True, download=True, transform=transforms.Compose([transforms.ToTensor(), normalize]))
    val_dataset = CIFAR10(root=root_dir, train=True, download=True, transform=transforms.Compose([transforms.ToTensor(), normalize]))


    train_sampler, val_sampler = samplers(len(train_dataset), split_shuffle, val_size)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
    val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=int(batch_size / 3))
    return train_dataloader, val_dataloader

def test_dataloader(root_dir='./cifar10', batch_size=120):
    test_dataset = CIFAR10(root=root_dir, train=False, download=True, transform=transforms.Compose([transforms.ToTensor(), normalize]))
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
    return test_dataloader

In [20]:
def train_batch(model, x_batch, y_batch, loss_fn, optimizer, presample=3.0):
    model.train()
    model.zero_grad()
    batch_size = x_batch.shape[0]
    selected_batch_size = int(batch_size / presample)
    output = model(x_batch)
    num_classes = output.shape[1]
    with torch.no_grad():
        probs = F.softmax(output, dim=1)
        one_hot_targets = F.one_hot(y_batch, num_classes=num_classes)
        g_i_norm = torch.norm(probs - one_hot_targets, dim=-1).detach().cpu().numpy()
    p_i = g_i_norm / np.sum(g_i_norm)
    batch_indices = np.random.choice(np.arange(batch_size), size=selected_batch_size, replace=True, p=p_i)
    selected_p_i = p_i[batch_indices]
    loss = loss_fn(output, y_batch)
    selected_loss = loss[batch_indices]
    w_i = 1.0 / (batch_size * selected_p_i)
    weighted_loss = (torch.tensor(w_i).to(selected_loss.device).detach() * selected_loss).mean()
    weighted_loss.backward()
    optimizer.step()
    batch_loss = loss.mean().cpu().item()
    weighted_batch_loss = weighted_loss.mean().cpu().item()
    max_p_i = np.max(p_i)
    num_unique_points = np.unique(batch_indices).size
    with torch.no_grad():
        batch_acc_sum = (output.argmax(dim=1) == y_batch).sum().cpu().item()
    return batch_loss, batch_acc_sum, weighted_batch_loss, max_p_i, num_unique_points, selected_batch_size

In [21]:
def train_epoch(model, dataloader, loss_fn, optimizer):
    epoch_loss = 0.0
    epoch_acc = 0
    epoch_weighted_loss = 0.0
    epoch_max_p_i_s = []
    epoch_num_unique_points_s = []
    epoch_size = 0
    epoch_weighted_size = 0
    for i_batch, (X_batch, y_batch) in enumerate(dataloader):
        batch_loss, batch_acc_sum, weighted_batch_loss, max_p_i, num_unique_points, selected_batch_size = train_batch(model, X_batch.to(model.device), y_batch.to(model.device), loss_fn, optimizer)
        epoch_size += len(X_batch)
        epoch_weighted_size += selected_batch_size
        epoch_loss += batch_loss * len(X_batch)
        epoch_acc += batch_acc_sum
        epoch_weighted_loss += weighted_batch_loss * selected_batch_size
        epoch_max_p_i_s.append(max_p_i)
        epoch_num_unique_points_s.append(num_unique_points)
    epoch_loss /= epoch_size
    epoch_acc /= epoch_size
    epoch_weighted_loss /= epoch_weighted_size
    return epoch_loss, epoch_acc, epoch_weighted_loss, epoch_max_p_i_s, epoch_num_unique_points_s


In [22]:
def train_full(model, train_dataloader, val_dataloader, loss_fn, optimizer, n_epochs, callback=None):
    epochs = tqdm(range(n_epochs), desc='Epochs', leave=True)
    for i_epoch in epochs:
        epoch_loss, epoch_acc, epoch_weighted_loss, epoch_max_p_i_s, epoch_num_unique_points_s = train_epoch(model, train_dataloader, loss_fn, optimizer)
        if callback is not None:
            cb_dict = callback(model, val_dataloader, loss_fn, epoch_loss, epoch_acc, epoch_weighted_loss, epoch_max_p_i_s, epoch_num_unique_points_s)
            epochs.set_postfix(cb_dict)



In [34]:
class CallBack:
    def __init__(self, eval_fn, name=None):
        self.eval_fn = eval_fn
        self.train_losses = []
        self.train_accs = []
        self.train_w_losses = []
        self.train_max_p_i = []
        self.train_num_unique_points = []
        self.val_losses = []
        self.val_accs = []

    def last_info(self):
        return {'loss_train': f'{self.train_losses[-1]}',
                'acc_train': f'{self.train_accs[-1]}',
                'w_loss_train': f'{self.train_w_losses[-1]}',
                'loss_val': f'{self.val_losses[-1]}',
                'acc_val': f'{self.val_accs[-1]}',
        }
    def __call__(self, model, val_dataloader, loss_fn,
                 epoch_loss=None, epoch_acc=None, epoch_weighted_loss=None, epoch_max_p_i_s=None, epoch_num_unique_points_s=None):
        self.train_losses.append(epoch_loss)
        self.train_accs.append(epoch_acc)
        self.train_w_losses.append(epoch_weighted_loss)
        self.train_max_p_i.append(epoch_max_p_i_s)
        self.train_num_unique_points.append(epoch_num_unique_points_s)
        loss_val, acc_val = self.eval_fn(model, val_dataloader, loss_fn)
        self.val_losses.append(loss_val)
        self.val_accs.append(acc_val)
        return self.last_info()


In [32]:
def evaluate(model, dataloader, loss_fn):
    model.eval()
    logits = []
    targets = []
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            output = model(X_batch.to(model.device)).cpu()
            logits.append(output)
            targets.append(y_batch)
    logits = torch.cat(logits)
    targets = torch.cat(targets)
    loss = loss_fn(logits, targets).mean().item()
    acc = (logits.argmax(dim=1) == targets).sum().item() / len(targets)
    return loss, acc



In [25]:
####ResNet
#class BasicBlock(nn.Module):
#    expansion = 1

#    def __init__(self, in_planes, planes, stride=1):
#        super(BasicBlock, self).__init__()
#        self.activation = F.relu
#        self.conv1 = nn.Conv2d(
#            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
#        self.bn1 = nn.BatchNorm2d(planes)
#        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
#                               stride=1, padding=1, bias=False)
#        self.bn2 = nn.BatchNorm2d(planes)

#        self.shortcut = nn.Sequential()
#        if stride != 1 or in_planes != self.expansion*planes:
#            self.shortcut = nn.Sequential(
#                nn.Conv2d(in_planes, self.expansion*planes,
#                          kernel_size=1, stride=stride, bias=False),
#                nn.BatchNorm2d(self.expansion*planes)
#            )

#    def forward(self, x):
#        out = self.activation(self.bn1(self.conv1(x)))
#        out = self.bn2(self.conv2(out))
#        out += self.shortcut(x)
#        out = self.activation(out)
#        return out


#class Bottleneck(nn.Module):
#    expansion = 4

#    def __init__(self, in_planes, planes, stride=1):
#        super(Bottleneck, self).__init__()
#        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
#        self.bn1 = nn.BatchNorm2d(planes)
#        self.conv2 = nn.Conv2d(
#            planes, planes, kernel_size=3, stride=stride, padding=1, bias=False
#        )
#        self.bn2 = nn.BatchNorm2d(planes)
#        self.conv3 = nn.Conv2d(
#            planes, self.expansion * planes, kernel_size=1, bias=False
#        )
#        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

#        self.shortcut = nn.Sequential()
#        if stride != 1 or in_planes != self.expansion * planes:
#            self.shortcut = nn.Sequential(
#                nn.Conv2d(
#                    in_planes,
#                    self.expansion * planes,
#                    kernel_size=1,
#                    stride=stride,
#                    bias=False,
#                ),
#                nn.BatchNorm2d(self.expansion * planes),
#            )

#    def forward(self, x):
#        out = F.relu(self.bn1(self.conv1(x)))
#        out = F.relu(self.bn2(self.conv2(out)))
#        out = self.bn3(self.conv3(out))
#        out += self.shortcut(x)
#        out = F.relu(out)
#        return out

In [26]:
#class ResNet(nn.Module):
#    def __init__(self, block, num_blocks, num_classes=10):
#        super(ResNet, self).__init__()
#        self.in_planes = 64
#        self.activation = F.relu
#        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
#                               stride=1, padding=1, bias=False)
#        self.bn1 = nn.BatchNorm2d(64)
#        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
#        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
#        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
#        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
#        self.linear = nn.Linear(512*block.expansion, num_classes)


#    def _make_layer(self, block, planes, num_blocks, stride):
#        strides = [stride] + [1]*(num_blocks-1)
#        layers = []
#        for stride in strides:
#            layers.append(block(self.in_planes, planes, self.activation, stride))
#            self.in_planes = planes * block.expansion
#        return nn.Sequential(*layers)

#    def forward(self, x):
#        out = self.activation(self.bn1(self.conv1(x)))
#        out = self.layer1(out)
#        out = self.layer2(out)
#        out = self.layer3(out)
#        out = self.layer4(out)
#        out = F.avg_pool2d(out, 4)
#        out = out.view(out.size(0), -1)
#        out = self.linear(out)
#        return out


In [27]:
#def ResNet18():
#    return ResNet(BasicBlock, [2, 2, 2, 2])


#def ResNet34():
#    return ResNet(BasicBlock, [3, 4, 6, 3])


#def ResNet50():
#    return ResNet(Bottleneck, [3, 4, 6, 3])

In [28]:
#######
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_planes,
                    self.expansion * planes,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(self.expansion * planes),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(
            planes, self.expansion * planes, kernel_size=1, bias=False
        )
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_planes,
                    self.expansion * planes,
                    kernel_size=1,
                    stride=stride,
                    bias=False,
                ),
                nn.BatchNorm2d(self.expansion * planes),
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])



In [29]:
### WideResNet

class BasicBlockW(nn.Module):
    def __init__(self, in_planes, out_planes, stride, drop_rate=0.0):
        super(BasicBlockW, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = drop_rate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                                                                padding=0, bias=False) or None

    def forward(self, x):
        if not self.equalInOut:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))

        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)


class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, dropRate=0.0):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate)

    @staticmethod
    def _make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate):
        layers = []
        for i in range(nb_layers):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, dropRate))
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.layer(x)


class WideResNet(nn.Module):
    def __init__(self, depth=28, num_classes=10, widen_factor=10, drop_rate=0.0):
        super(WideResNet, self).__init__()
        n_channels = [16, 16 * widen_factor, 32 * widen_factor, 64 * widen_factor]
        assert ((depth - 4) % 6 == 0)
        n = int((depth - 4) / 6)
        block = BasicBlockW

        self.conv1 = nn.Conv2d(3, n_channels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)

        self.block1 = NetworkBlock(n, n_channels[0], n_channels[1], block, 1, drop_rate)

        self.block2 = NetworkBlock(n, n_channels[1], n_channels[2], block, 2, drop_rate)

        self.block3 = NetworkBlock(n, n_channels[2], n_channels[3], block, 2, drop_rate)

        self.bn1 = nn.BatchNorm2d(n_channels[3])
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(n_channels[3], num_classes)
        self.nChannels = n_channels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()

    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        return self.fc(out)


In [30]:

train_dataloader, val_dataloader = train_val_dataloader()
test_loader = test_dataloader()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
model = ResNet50()
model.to(device)
model.device = device
loss_fn = nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
cb = CallBack(evaluate)
train_full(model, train_dataloader, val_dataloader, loss_fn, optimizer, n_epochs=30, callback=cb)

Epochs:   0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
loss, acc = evaluate(model, test_loader, loss_fn)
print(f'ResNet50, test loss: {loss}')
print(f'ResNet50, test accuracy: {acc}')

In [ ]:
plt.figure(figsize=(15, 10))
plt.title('ResNet50, train and validation loss')
plt.xlabel('Number of epoch')
plt.ylabel('Loss')
epochs = np.arange(30) + 1
plt.plot(epochs, cb.train_losses, label='Train')
plt.plot(epochs, cb.train_w_losses, label='Train weighted')
plt.plot(epochs, cb.val_losses, label='Validation')
plt.legend()
plt.grid(True)


In [ ]:
print(f'Best loss on train: {np.min(cb.train_losses)}, on {np.argmin(cb.train_losses) + 1} epoch')
print(f'Best weighted loss on train: {np.min(cb.train_w_losses)}, on {np.argmin(cb.train_w_losses) + 1} epoch')
print(f'Best loss on validation: {np.min(cb.val_losses)}, on {np.argmin(cb.val_losses) + 1} epoch')

In [ ]:
plt.figure(figsize=(15, 10))
plt.title('ResNet50, train and validation accuracy')
plt.xlabel('Number of epoch')
plt.ylabel('Accuracy')
epochs = np.arange(30) + 1
plt.plot(epochs, cb.train_accs, label='Train')
plt.plot(epochs, cb.val_accs, label='Validation')
plt.legend()
plt.grid(True)

In [ ]:
print(f'Best accuracy on train: {np.max(cb.train_accs)}, on {np.argmax(cb.train_accs) + 1} epoch')
print(f'Best accuracy on validation: {np.max(cb.val_accs)}, on {np.argmax(cb.val_accs) + 1} epoch')